# Data Mover Notebook 
## Prerequisites:
- Python 3.8
- PyCelonis 2.0 or later

In [ ]:
#Please Note: If you see the following error in your console after each exercise, you may simply ignore it:

#Your PyCelonis Version 2.11.0 is outdated (Newest Version: 2.11.1). Please upgrade the package via: pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis pycelonis_core --upgrade


#Steps for this module

#1.run the following command inside of your terminal - !python3.8 -m pip install --upgrade pip 

!python3.8 -m pip install --upgrade pip 
!pip install --upgrade pycelonis

from pathlib import Path 
from pycelonis import get_celonis 
from pycelonis.pql import PQL, PQLColumn 
from pycelonis.ems import ExportType, JobType
import os, pycelonis


#If you followed this tutorial correctly, you should see something like the following printed in your console after running this module:

#PyCelonis version: 2.11.0

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
ERROR: Could not find a version that satisfies the requirement pycelonis (from versions: none)
ERROR: No matching distribution found for pycelonis
PyCelonis version: 2.11.0


## Step 1: Define Environment Variables
Replace the placeholders with your own details for the source and target environments.


In [ ]:
# Required imports
import os
from pycelonis import get_celonis
from pathlib import Path

# Define the necessary configuration variables directly in the code
source_data_pool_id = "id of the source data pool that you are going to use to pull data from"
source_data_model_id = "id of the source data model"
source_url = "source Celonis URL"
source_api_token = "source Celonis API token"
target_data_pool_id = "id of the target data pool where the data will be moved"
target_data_model_id = "id of the target data model"
target_url = "target Celonis URL"
target_api_token = "target Celonis API token"
data_dir = "directory that you want to save the tables"

# Initialize the Celonis connection
c = get_celonis(url=source_url, api_token=source_api_token)

# Define the list of tables to pull
tables_to_pull = ["newairqualityparquet", "another_table_name", "more_table_names"]

# Directory to save pulled data
data_dir = Path(data_dir)
data_dir.mkdir(parents=True, exist_ok=True)

# Print the data directory path (optional)
print(f"Data will be saved to: {data_dir}")


Data directory: /Users/o.wu/owenfeature/pycelonis-examples/datamoverproject/data


## Step 2: Download data from source environment


In [ ]:
# Import necessary libraries
from pycelonis import get_celonis
from pathlib import Path

# Define your source data pool and model details directly (instead of using a config file)
source_url = "https://your-celonis-instance-url"
source_api_token = "your-api-token"
source_data_pool_id = "id_of_the_source_data_pool"
source_data_model_id = "id_of_the_source_data_model"

# Connect to the source Celonis instance using the provided URL and API token
celonis_from = get_celonis(base_url=source_url, api_token=source_api_token, permissions=False, key_type='APP_KEY')

# Retrieve the data pool and data model
datapool_from = celonis_from.data_integration.get_data_pool(source_data_pool_id)
datamodel_from = datapool_from.get_data_model(source_data_model_id)

# Get all the tables in the data model
datamodel_from_tables = datamodel_from.get_tables()

# Print the names of all the tables in the data model
print("Tables in this datamodel:")
for table in datamodel_from_tables:
    print(table.name)


Your PyCelonis Version 2.11.0 is outdated (Newest Version: 2.11.1). Please upgrade the package via: pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis pycelonis_core --upgrade


Tables in this datamodel:
newairqualityparquet


## Step 3: Insert the tables you want to download

In [ ]:
from pathlib import Path

# Directly define the list of tables you want to download (instead of reading from a file)
tables_to_download = ["newairqualityparquet"]

tables_from_detail = []

# Assuming datamodel_from_tables is already defined
for table_name in tables_to_download:
    table = datamodel_from_tables.find(table_name)
    if table:
        tables_from_detail.append(table)
        print("Added table:", table.name)
    else:
        print(f"Table '{table_name}' not found in datamodel.")

# Define the directory for saving parquet files
data_dir = Path("./data")
data_dir.mkdir(parents=True, exist_ok=True)

# Function to download table data and save as Parquet
def download_table_data(table):
    table_name = table.name
    query = PQL(distinct=False, limit=None)
    for column in table.get_columns():
        col_name = column.name
        if col_name != '_CELONIS_CHANGE_DATE':  # Skip the _CELONIS_CHANGE_DATE column
            query += PQLColumn(name=col_name, query=f'"{table_name}"."{col_name}"')

    data_export = datamodel_from.create_data_export(query=query, export_type=ExportType.PARQUET)
    data_export.wait_for_execution()

    for i, chunk in enumerate(data_export.get_chunks()):
        file_path = data_dir / f"{table_name}_{i}.parquet"
        with open(file_path, "wb") as file:
            file.write(chunk.read())
        print(f"Downloaded table: {table_name}, chunk: {i}")

# Download the data for each table in the list
for table in tables_from_detail:
    download_table_data(table)

# Final message after completion
print("Conversion to Parquet completed.")

Added table:  newairqualityparquet
name='datetime' length=19 type_='STRING' client=<pycelonis_core.client.client.Client object at 0x137c55b80> data_pool_id='7873300c-ba33-4c61-ac8f-4d912f7c0312' data_model_id='cd425ef4-6815-4360-b155-388d54eea294' table_name='newairqualityparquet' table_alias=None
name='station_antwerp' length=15 type_='FLOAT' client=<pycelonis_core.client.client.Client object at 0x137c55b80> data_pool_id='7873300c-ba33-4c61-ac8f-4d912f7c0312' data_model_id='cd425ef4-6815-4360-b155-388d54eea294' table_name='newairqualityparquet' table_alias=None
name='station_paris' length=15 type_='FLOAT' client=<pycelonis_core.client.client.Client object at 0x137c55b80> data_pool_id='7873300c-ba33-4c61-ac8f-4d912f7c0312' data_model_id='cd425ef4-6815-4360-b155-388d54eea294' table_name='newairqualityparquet' table_alias=None
name='station_london' length=15 type_='FLOAT' client=<pycelonis_core.client.client.Client object at 0x137c55b80> data_pool_id='7873300c-ba33-4c61-ac8f-4d912f7c0312

## Step 6: Upload Data to Target Environment


In [ ]:
from pycelonis import get_celonis

# Define the local path for your Parquet file
parquet_file_path = "/Users/o.wu/Downloads/newairqualityparquet.parquet"

# Define the target Celonis environment
celonis_to = get_celonis(base_url=target_url, api_token=target_api_token, permissions=False, key_type='APP_KEY')
datapool_to = celonis_to.data_integration.get_data_pool(target_data_pool_id)

# Define the table names you want to upload
table_names = ["newairqualityparquet"]

# Upload each table to Celonis
for table in table_names:
    data_push_job = datapool_to.create_data_push_job(target_name=table, type_=JobType.REPLACE)
    
    # Open the Parquet file and add it to the data push job
    with open(parquet_file_path, "rb") as file:
        data_push_job.add_file_chunk(file)
        data_push_job.execute(wait=True)
    
    print(f"Uploaded table: {table}")

# Final message after completion
print("Parquet upload completed.")

Your PyCelonis Version 2.11.0 is outdated (Newest Version: 2.11.1). Please upgrade the package via: pip install --extra-index-url=https://pypi.celonis.cloud/ pycelonis pycelonis_core --upgrade


28050e78-1a20-417e-9f21-c6185c1505ba
Uploaded table: newairqualityparquet


## Notes
- **User Instructions**: Update variables in each section as per your environment’s requirements.
- **Modularity**: Functions for downloading and uploading tables improve reusability.
- **Logs and Feedback**: Each major step includes feedback, so the user knows the status of each operation.
